In [2]:
#xrs时间列表记录，分辨率不同，需要分开运行
import datetime
import time
import xlrd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as spio   #用来读取sav格式
import numpy as np 
import csv 
#from scipy import interpolate
import scipy
sheet_xrs_time = pd.read_csv('xrs_2010-2016_time.csv')
sheet_lya_time = pd.read_csv('lya_2010-2016_time_change_1.csv')
l_index=range(100)
sheet3_x_time = pd.DataFrame()
sheet3_x_value = pd.DataFrame()
x_num_list=[] #把对应的中断的位置点记录下来，便于和后面的数值对应，不能直接记录时间，需要记录m
#list_lya_peak=[]
list_xrs_start=[]
list_x_value=[]
list_year=[]
for i in range(len(sheet_lya_time['DATE'])):
    a=sheet_lya_time['DATE'][i].split('-')[0]+sheet_lya_time['DATE'][i].split('-')[1]+sheet_lya_time['DATE'][i].split('-')[2]
    if(a=='20150819'):#中断点，不记录
        print('20150819跳过数据')
        continue
    
    list_year.append(sheet_lya_time['DATE'][i].split('-')[0]+sheet_lya_time['DATE'][i].split('-')[1]+sheet_lya_time['DATE'][i].split('-')[2])
    #list_lya_peak.append(int(sheet_lya_time['PEAKT'][i][0:2])*3600+int(sheet_lya_time['PEAKT'][i][3:5])*60+int(sheet_lya_time['PEAKT'][i][6:8]))
    list_xrs_start.append(int(sheet_xrs_time['START'][i][0:2])*3600+int(sheet_xrs_time['START'][i][3:5])*60+int(sheet_xrs_time['START'][i][6:8]))
#sheet_xrs = pd.DataFrame()
k=0
n=0
for  i in list_year:
    try: #处理文件不存在的情况
        print(i)
        if(n<list_year.index('20101019')):#2010年10月19日后的数据是g15
            sheet_xrs = spio.readsav('/home/tzy/flare_data/sav/lc'+i+'_xrs_g14.sav',python_dict=True, verbose=False)
        else:
            sheet_xrs = spio.readsav('/home/tzy/flare_data/sav/lc'+i+'_xrs_g15.sav',python_dict=True, verbose=False)
    except:
        print('该项数据文件缺少：',i)
        continue
    m=0
    break_out_flag = False #跳出整个循环的标志    
    b=[]
    for j in range(len(sheet_xrs['tsecs'])):
        sheet3_x_time.loc[m,n]=sheet_xrs['tsecs'][j]-sheet_xrs['tsecs'][0]#取一个当天最开始时间戳，后面的时间减这个时间得到的就是0时以后的秒数，可以转为时   
        sheet3_x_value.loc[m,n]=sheet_xrs['yclean'][0][j]#yclean的数据有两列，选第一列
        if((list_xrs_start[n]-sheet3_x_time.loc[m,n])<10): #取一个阈值
            
            print('x背景值中断点:',sheet3_x_value.loc[m,n])
            break_out_flag = True #满足条件，标记为True
            list_x_value.append(sheet3_x_value.loc[m,n])
            break   #选择在耀斑结束后一小时结束循环
        m=m+1;
    n=n+1;
    if(break_out_flag): #这里用了一个标志位一次性跳出嵌套循环
        x_num_list.append(m)
        print('中断位置已存入列表：',m)
    else:
        x_num_list.append(m-1)
        print('非中断位置：',m-1)

20150819跳过数据
20100409
x峰值中断点: 17031.0
中断位置已存入列表： 8316
20100609
x峰值中断点: 31131.0
中断位置已存入列表： 15201
20100731
x峰值中断点: 18891.0
中断位置已存入列表： 9224
20101019
x峰值中断点: 47930.800999999046
中断位置已存入列表： 23399
20101023
x峰值中断点: 62930.17999994755
中断位置已存入列表： 30728
20101026
x峰值中断点: 11450.256999969482
中断位置已存入列表： 5591
20101110
x峰值中断点: 59630.69099998474
中断位置已存入列表： 29117
20101116
x峰值中断点: 10671.980000019073
中断位置已存入列表： 5200
20110104
x峰值中断点: 34971.10199999809
中断位置已存入列表： 17045
20110126
x峰值中断点: 79311.71899998188
中断位置已存入列表： 38254
20110211
x峰值中断点: 73790.34399998188
中断位置已存入列表： 36031
20110212
x峰值中断点: 38811.01999998093
中断位置已存入列表： 18951
20110325
x峰值中断点: 10430.330000042915
中断位置已存入列表： 5093
20110329
x峰值中断点: 1611.7560000419617
中断位置已存入列表： 787
20110331
x峰值中断点: 77691.64800000191
中断位置已存入列表： 37936
20110501
x峰值中断点: 35331.56299996376
中断位置已存入列表： 17252
20110616
x峰值中断点: 83391.24199998379
中断位置已存入列表： 40719
20110810
x峰值中断点: 48350.52300000191
中断位置已存入列表： 23609
20110907
x峰值中断点: 66231.28100001812
中断位置已存入列表： 32340
20111002
x峰值中断点: 69170.10199999

In [11]:
x_value= pd.DataFrame() #记录导数峰值差

for i in range(99):
    x_value.loc[i,0]=sheet3_x_value.loc[x_num_list[i],i]

In [13]:
x_value.to_csv('x_start_value.csv');